# Comprehensive Training Pipeline

1. **1**: Train YOLO11n on all folds of `datasets/k_fold_cv`
2. **2**: Train YOLO11n, s, and m on all folds of `datasets/k_fold_cv_augmented`
3. **3**: Train YOLO11l and x on first fold of `datasets/k_fold_cv_augmented`
4. **4**: Train RT-DETR-l and x on first fold of `datasets/k_fold_cv_augmented`

In [1]:
# Install required packages
!pip install -U ultralytics torch

In [2]:
# Import required libraries
import torch
import os
import yaml
import json
import datetime
import psutil
from ultralytics import YOLO, RTDETR
from pathlib import Path

print('Torch:', torch.__version__, '| CUDA:', torch.version.cuda, '| Device count:', torch.cuda.device_count())
if torch.cuda.is_available():
    print('GPU:', torch.cuda.get_device_name(0))
print('RAM:', f'{psutil.virtual_memory().total/1e9:.1f} GB')

# Enable CUDNN benchmarking for faster training
torch.backends.cudnn.benchmark = True

Torch: 2.7.1+cu126 | CUDA: 12.6 | Device count: 1
GPU: Tesla T4
RAM: 270.0 GB


In [3]:
# Configuration and Helper Functions

class TrainingConfig:
    """Configuration class for training parameters"""
    
    # Dataset paths - using absolute paths
    K_FOLD_CV_PATH = "/home/andrea/work/AI-waste-detection/datasets/k_fold_cv"
    K_FOLD_CV_AUGMENTED_PATH = "/home/andrea/work/AI-waste-detection/datasets/k_fold_cv_augmented"
    
    # Training parameters
    EPOCHS = 100
    PATIENCE = 20
    IMGSZ = 640
    BATCH_SIZE = 32
    WORKERS = 8
    
    # Model configurations
    YOLO_MODELS = {
        'yolo11n': 'yolo11n.pt',
        'yolo11s': 'yolo11s.pt', 
        'yolo11m': 'yolo11m.pt',
        'yolo11l': 'yolo11l.pt',
        'yolo11x': 'yolo11x.pt'
    }
    
    RTDETR_MODELS = {
        'rtdetr-l': 'rtdetr-l.pt',
        'rtdetr-x': 'rtdetr-x.pt'
    }

def get_training_params(model_size):
    """Get training parameters based on model size"""
    base_params = {
        'epochs': TrainingConfig.EPOCHS,
        'imgsz': TrainingConfig.IMGSZ,
        'patience': TrainingConfig.PATIENCE,
        'amp': True,
        'optimizer': 'AdamW',
        'save_period': 10,
        'workers': TrainingConfig.WORKERS,
        'cache': True,
        'half': True,
        'pretrained': True,
        'lr0': 1e-4,
        'lrf': 0.01,
        'momentum': 0.937,
        'weight_decay': 1e-4,
        'augment': False
    }
    
    # Adjust batch size based on model size
    if model_size in ['yolo11n', 'yolo11s']:
        base_params['batch'] = 32
    elif model_size in ['yolo11m', 'rtdetr-l']:
        base_params['batch'] = 16
    elif model_size in ['yolo11l', 'yolo11x', 'rtdetr-x']:
        base_params['batch'] = 8
    
    return base_params

def save_results(results, filename):
    """Save training results to JSON file"""
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filepath = f"training_results_{filename}_{timestamp}.json"
    
    with open(filepath, 'w') as f:
        json.dump(results, f, indent=2, default=str)
    
    print(f"Results saved to: {filepath}")
    return filepath

def train_model(model_type, model_name, data_yaml, fold_idx, experiment_name, project_dir="runs"):
    """Train a single model on a specific fold"""
    
    print(f"\n{'='*80}")
    print(f"Training {model_name} on Fold {fold_idx}")
    print(f"Data: {data_yaml}")
    print(f"{'='*80}")
    
    try:
        # Load model
        if model_type == 'yolo':
            model_path = TrainingConfig.YOLO_MODELS[model_name]
            model = YOLO(model_path)
        elif model_type == 'rtdetr':
            model_path = TrainingConfig.RTDETR_MODELS[model_name]
            model = RTDETR(model_path)
        else:
            raise ValueError(f"Unknown model type: {model_type}")
        
        # Get training parameters
        train_params = get_training_params(model_name)
        
        # Set training-specific parameters
        train_params.update({
            'data': data_yaml,
            'name': f"{model_name}_fold_{fold_idx}",
            'project': f"{project_dir}/{experiment_name}"
        })
        
        print(f"Training parameters: {train_params}")
        
        # Train the model
        results = model.train(**train_params)
        
        # Extract metrics
        fold_result = {
            'model_type': model_type,
            'model_name': model_name,
            'fold': fold_idx,
            'status': 'completed',
            'best_epoch': results.best_epoch if hasattr(results, 'best_epoch') else None,
            'best_fitness': results.best_fitness if hasattr(results, 'best_fitness') else None,
            'model_path': model.ckpt_path if hasattr(model, 'ckpt_path') else None,
            'experiment_name': experiment_name,
            'data_yaml': data_yaml,
            'training_params': train_params
        }
        
        print(f"\n✅ {model_name} Fold {fold_idx} completed successfully!")
        print(f"Best epoch: {fold_result['best_epoch']}")
        print(f"Best fitness: {fold_result['best_fitness']:.4f}" if fold_result['best_fitness'] else "Best fitness: N/A")
        print(f"Model saved: {fold_result['model_path']}")
        
        return fold_result
        
    except Exception as e:
        print(f"❌ Error training {model_name} on fold {fold_idx}: {str(e)}")
        return {
            'model_type': model_type,
            'model_name': model_name,
            'fold': fold_idx,
            'status': 'failed',
            'error': str(e),
            'experiment_name': experiment_name,
            'data_yaml': data_yaml
        }

print("Configuration and helper functions loaded!")

Configuration and helper functions loaded!


In [4]:
# Verify Dataset Structure

print("\n" + "="*100)
print("DATASET VERIFICATION")
print("="*100)

def verify_dataset_structure(dataset_path, dataset_name):
    """Verify the structure of a k-fold dataset"""
    print(f"\n📁 Verifying {dataset_name}: {dataset_path}")
    
    if not os.path.exists(dataset_path):
        print(f"❌ Dataset path not found: {dataset_path}")
        return False
    
    all_folds_valid = True
    
    for fold_idx in range(5):
        fold_dir = os.path.join(dataset_path, f"fold_{fold_idx}")
        data_yaml = os.path.join(fold_dir, "data.yaml")
        
        if os.path.exists(data_yaml):
            # Count images in train and val
            train_images = os.path.join(fold_dir, "train", "images")
            val_images = os.path.join(fold_dir, "val", "images")
            
            train_count = len([f for f in os.listdir(train_images) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]) if os.path.exists(train_images) else 0
            val_count = len([f for f in os.listdir(val_images) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]) if os.path.exists(val_images) else 0
            
            total = train_count + val_count
            
            if total > 0:
                print(f"  ✅ Fold {fold_idx}: Train={train_count}, Val={val_count}, Total={total}")
            else:
                print(f"  ❌ Fold {fold_idx}: No images found")
                all_folds_valid = False
        else:
            print(f"  ❌ Fold {fold_idx}: data.yaml not found")
            all_folds_valid = False
    
    return all_folds_valid

# Verify both datasets
k_fold_cv_valid = verify_dataset_structure(TrainingConfig.K_FOLD_CV_PATH, "k_fold_cv")
k_fold_cv_augmented_valid = verify_dataset_structure(TrainingConfig.K_FOLD_CV_AUGMENTED_PATH, "k_fold_cv_augmented")

print(f"\n📊 VERIFICATION SUMMARY:")
print(f"k_fold_cv: {'✅ Valid' if k_fold_cv_valid else '❌ Issues found'}")
print(f"k_fold_cv_augmented: {'✅ Valid' if k_fold_cv_augmented_valid else '❌ Issues found'}")

if k_fold_cv_valid and k_fold_cv_augmented_valid:
    print(f"\n🚀 All datasets verified! Ready to start training pipeline.")
else:
    print(f"\n⚠️ Please fix dataset issues before proceeding with training.")


DATASET VERIFICATION

📁 Verifying k_fold_cv: /home/andrea/work/AI-waste-detection/datasets/k_fold_cv
  ✅ Fold 0: Train=3488, Val=873, Total=4361
  ✅ Fold 1: Train=3489, Val=872, Total=4361
  ✅ Fold 2: Train=3489, Val=872, Total=4361
  ✅ Fold 3: Train=3489, Val=872, Total=4361
  ✅ Fold 4: Train=3489, Val=872, Total=4361

📁 Verifying k_fold_cv_augmented: /home/andrea/work/AI-waste-detection/datasets/k_fold_cv_augmented
  ✅ Fold 0: Train=12864, Val=3217, Total=16081
  ✅ Fold 1: Train=12865, Val=3216, Total=16081
  ✅ Fold 2: Train=12865, Val=3216, Total=16081
  ✅ Fold 3: Train=12865, Val=3216, Total=16081
  ✅ Fold 4: Train=12865, Val=3216, Total=16081

📊 VERIFICATION SUMMARY:
k_fold_cv: ✅ Valid
k_fold_cv_augmented: ✅ Valid

🚀 All datasets verified! Ready to start training pipeline.


In [5]:
# Stage 1: Train YOLO11n on all folds of k_fold_cv

print("\n" + "="*100)
print("STAGE 1: Training YOLO11n on all folds of k_fold_cv")
print("="*100)

stage1_results = []
k_fold_cv_path = TrainingConfig.K_FOLD_CV_PATH

# Check if dataset exists
if not os.path.exists(k_fold_cv_path):
    print(f"❌ Dataset path not found: {k_fold_cv_path}")
else:
    print(f"📁 Dataset path: {k_fold_cv_path}")
    
    # Train on all 5 folds (0-4)
    for fold_idx in range(5):
        fold_dir = os.path.join(k_fold_cv_path, f"fold_{fold_idx}")
        data_yaml = os.path.join(fold_dir, "data.yaml")
        
        if os.path.exists(data_yaml):
            result = train_model(
                model_type='yolo',
                model_name='yolo11n',
                data_yaml=data_yaml,
                fold_idx=fold_idx,
                experiment_name='stage1_yolo11n_k_fold_cv'
            )
            stage1_results.append(result)
        else:
            print(f"❌ Data YAML not found: {data_yaml}")
            stage1_results.append({
                'model_name': 'yolo11n',
                'fold': fold_idx,
                'status': 'failed',
                'error': f'Data YAML not found: {data_yaml}'
            })

# Save Stage 1 results
stage1_file = save_results(stage1_results, "stage1")

# Summary
successful_stage1 = [r for r in stage1_results if r['status'] == 'completed']
print(f"\n📊 Stage 1 Summary: {len(successful_stage1)}/5 folds completed successfully")


STAGE 1: Training YOLO11n on all folds of k_fold_cv
📁 Dataset path: /home/andrea/work/AI-waste-detection/datasets/k_fold_cv

Training yolo11n on Fold 0
Data: /home/andrea/work/AI-waste-detection/datasets/k_fold_cv/fold_0/data.yaml
Training parameters: {'epochs': 100, 'imgsz': 640, 'patience': 20, 'amp': True, 'optimizer': 'AdamW', 'save_period': 10, 'workers': 8, 'cache': True, 'half': True, 'pretrained': True, 'lr0': 0.0001, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0001, 'augment': False, 'batch': 32, 'data': '/home/andrea/work/AI-waste-detection/datasets/k_fold_cv/fold_0/data.yaml', 'name': 'yolo11n_fold_0', 'project': 'runs/stage1_yolo11n_k_fold_cv'}
Ultralytics 8.3.167 🚀 Python-3.11.0rc1 torch-2.7.1+cu126 CUDA:0 (Tesla T4, 14914MiB)
Training parameters: {'epochs': 100, 'imgsz': 640, 'patience': 20, 'amp': True, 'optimizer': 'AdamW', 'save_period': 10, 'workers': 8, 'cache': True, 'half': True, 'pretrained': True, 'lr0': 0.0001, 'lrf': 0.01, 'momentum': 0.937, 'weight_deca

train: Scanning /home/andrea/work/AI-waste-detection/datasets/k_fold_cv/fold_0/train/labels.cache... 3488 images, 

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.



train: Caching images (3.0GB RAM): 100%|██████████| 3488/3488 [00:00<00:00, 4155.41it/s]



albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 582.2±304.6 MB/s, size: 22.6 KB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 582.2±304.6 MB/s, size: 22.6 KB)


...

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.



val: Caching images (0.8GB RAM): 100%|██████████| 873/873 [00:00<00:00, 3904.12it/s]


Plotting labels to runs/stage1_yolo11n_k_fold_cv/yolo11n_fold_0/labels.jpg... 
optimizer: AdamW(lr=0.0001, momentum=0.937) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0001), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/stage1_yolo11n_k_fold_cv/yolo11n_fold_0
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
optimizer: AdamW(lr=0.0001, momentum=0.937) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0001), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/stage1_yolo11n_k_fold_cv/yolo11n_fold_0
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      4.35G      1.149      2.415      1.486        131        640:  99%|█████████▉| 108/109 [00:35<00:



KeyboardInterrupt: 

In [5]:
# Stage 2: Train YOLO11n, s, and m on all folds of k_fold_cv_augmented

print("\n" + "="*100)
print("STAGE 2: Training YOLO11n, s, and m on all folds of k_fold_cv_augmented")
print("="*100)

stage2_results = []
k_fold_cv_augmented_path = TrainingConfig.K_FOLD_CV_AUGMENTED_PATH
stage2_models = ['yolo11n', 'yolo11s', 'yolo11m']

# Check if dataset exists
if not os.path.exists(k_fold_cv_augmented_path):
    print(f"❌ Dataset path not found: {k_fold_cv_augmented_path}")
else:
    print(f"📁 Dataset path: {k_fold_cv_augmented_path}")
    
    # Train each model on all 5 folds (0-4)
    for model_name in stage2_models:
        print(f"\n🚀 Starting {model_name} training on all folds...")
        
        for fold_idx in range(5):
            fold_dir = os.path.join(k_fold_cv_augmented_path, f"fold_{fold_idx}")
            data_yaml = os.path.join(fold_dir, "data.yaml")
            
            if os.path.exists(data_yaml):
                result = train_model(
                    model_type='yolo',
                    model_name=model_name,
                    data_yaml=data_yaml,
                    fold_idx=fold_idx,
                    experiment_name=f'stage2_{model_name}_k_fold_cv_augmented'
                )
                stage2_results.append(result)
            else:
                print(f"❌ Data YAML not found: {data_yaml}")
                stage2_results.append({
                    'model_name': model_name,
                    'fold': fold_idx,
                    'status': 'failed',
                    'error': f'Data YAML not found: {data_yaml}'
                })

# Save Stage 2 results
stage2_file = save_results(stage2_results, "stage2")

# Summary
successful_stage2 = [r for r in stage2_results if r['status'] == 'completed']
total_expected = len(stage2_models) * 5  # 3 models × 5 folds = 15
print(f"\n📊 Stage 2 Summary: {len(successful_stage2)}/{total_expected} trainings completed successfully")

# Per-model summary
for model_name in stage2_models:
    model_results = [r for r in stage2_results if r['model_name'] == model_name and r['status'] == 'completed']
    print(f"  {model_name}: {len(model_results)}/5 folds completed")


STAGE 2: Training YOLO11n, s, and m on all folds of k_fold_cv_augmented
📁 Dataset path: /home/andrea/work/AI-waste-detection/datasets/k_fold_cv_augmented

🚀 Starting yolo11n training on all folds...

Training yolo11n on Fold 0
Data: /home/andrea/work/AI-waste-detection/datasets/k_fold_cv_augmented/fold_0/data.yaml
Training parameters: {'epochs': 100, 'imgsz': 640, 'patience': 20, 'amp': True, 'optimizer': 'AdamW', 'save_period': 10, 'workers': 8, 'cache': True, 'half': True, 'pretrained': True, 'lr0': 0.0001, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0001, 'augment': False, 'batch': 32, 'data': '/home/andrea/work/AI-waste-detection/datasets/k_fold_cv_augmented/fold_0/data.yaml', 'name': 'yolo11n_fold_0', 'project': 'runs/stage2_yolo11n_k_fold_cv_augmented'}
Ultralytics 8.3.167 🚀 Python-3.11.0rc1 torch-2.7.1+cu126 CUDA:0 (Tesla T4, 14914MiB)
Training parameters: {'epochs': 100, 'imgsz': 640, 'patience': 20, 'amp': True, 'optimizer': 'AdamW', 'save_period': 10, 'workers': 8, 'ca

train: Scanning /home/andrea/work/AI-waste-detection/datasets/k_fold_cv_augmented/fold_0/train/labels... 12864 ima



train: New cache created: /home/andrea/work/AI-waste-detection/datasets/k_fold_cv_augmented/fold_0/train/labels.cache
WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.
WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (12.3GB RAM): 100%|██████████| 12864/12864 [00:03<00:00, 3473.85it/s]



albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 653.1±407.5 MB/s, size: 24.8 KB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 653.1±407.5 MB/s, size: 24.8 KB)


...

val: New cache created: /home/andrea/work/AI-waste-detection/datasets/k_fold_cv_augmented/fold_0/val/labels.cache
WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.
WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


val: Caching images (3.1GB RAM): 100%|██████████| 3217/3217 [00:00<00:00, 3634.05it/s]


Plotting labels to runs/stage2_yolo11n_k_fold_cv_augmented/yolo11n_fold_0/labels.jpg... 
optimizer: AdamW(lr=0.0001, momentum=0.937) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0001), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/stage2_yolo11n_k_fold_cv_augmented/yolo11n_fold_0
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
optimizer: AdamW(lr=0.0001, momentum=0.937) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0001), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/stage2_yolo11n_k_fold_cv_augmented/yolo11n_fold_0
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      4.42G      1.121      2.661       1.44        148        640:  19%|█▉        | 77/402 [00:26<01:5



KeyboardInterrupt: 

In [ ]:
# Stage 3: Train YOLO11l and x on first fold of k_fold_cv_augmented

print("\n" + "="*100)
print("STAGE 3: Training YOLO11l and x on first fold of k_fold_cv_augmented")
print("="*100)

stage3_results = []
stage3_models = ['yolo11l', 'yolo11x']
fold_idx = 0  # First fold

# Check if dataset exists
if not os.path.exists(k_fold_cv_augmented_path):
    print(f"❌ Dataset path not found: {k_fold_cv_augmented_path}")
else:
    fold_dir = os.path.join(k_fold_cv_augmented_path, f"fold_{fold_idx}")
    data_yaml = os.path.join(fold_dir, "data.yaml")
    
    if os.path.exists(data_yaml):
        print(f"📁 Using fold {fold_idx}: {data_yaml}")
        
        # Train each large model on fold 0
        for model_name in stage3_models:
            print(f"\n🚀 Starting {model_name} training on fold {fold_idx}...")
            
            result = train_model(
                model_type='yolo',
                model_name=model_name,
                data_yaml=data_yaml,
                fold_idx=fold_idx,
                experiment_name=f'stage3_{model_name}_fold0_augmented'
            )
            stage3_results.append(result)
    else:
        print(f"❌ Data YAML not found: {data_yaml}")
        for model_name in stage3_models:
            stage3_results.append({
                'model_name': model_name,
                'fold': fold_idx,
                'status': 'failed',
                'error': f'Data YAML not found: {data_yaml}'
            })

# Save Stage 3 results
stage3_file = save_results(stage3_results, "stage3")

# Summary
successful_stage3 = [r for r in stage3_results if r['status'] == 'completed']
print(f"\n📊 Stage 3 Summary: {len(successful_stage3)}/{len(stage3_models)} large YOLO models completed successfully")


STAGE 3: Training YOLO11l and x on first fold of k_fold_cv_augmented
📁 Using fold 0: /home/andrea/work/AI-waste-detection/datasets/k_fold_cv_augmented/fold_0/data.yaml

🚀 Starting yolo11l training on fold 0...

Training yolo11l on Fold 0
Data: /home/andrea/work/AI-waste-detection/datasets/k_fold_cv_augmented/fold_0/data.yaml


100%|████████████████████████████████████████████████████████████████████████| 49.0M/49.0M [00:00<00:00, 63.5MB/s]



Training parameters: {'epochs': 100, 'imgsz': 640, 'patience': 20, 'amp': True, 'optimizer': 'AdamW', 'save_period': 10, 'workers': 8, 'cache': True, 'half': True, 'pretrained': True, 'lr0': 0.0001, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0001, 'augment': False, 'batch': 8, 'data': '/home/andrea/work/AI-waste-detection/datasets/k_fold_cv_augmented/fold_0/data.yaml', 'name': 'yolo11l_fold_0', 'project': 'runs/stage3_yolo11l_fold0_augmented'}
Ultralytics 8.3.167 🚀 Python-3.11.0rc1 torch-2.7.1+cu126 CUDA:0 (Tesla T4, 14914MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/andrea/work/AI-waste-detection/datasets/k_fold_cv_augmented/fold_0/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, er

train: Scanning /home/andrea/work/AI-waste-detection/datasets/k_fold_cv_augmented/fold_0/train/labels.cache... 128



WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


...

val: Scanning /home/andrea/work/AI-waste-detection/datasets/k_fold_cv_augmented/fold_0/val/labels.cache... 3217 im

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.



val: Caching images (3.1GB RAM): 100%|██████████| 3217/3217 [00:01<00:00, 2873.36it/s]



Plotting labels to runs/stage3_yolo11l_fold0_augmented/yolo11l_fold_0/labels.jpg... 
optimizer: AdamW(lr=0.0001, momentum=0.937) with parameter groups 167 weight(decay=0.0), 174 weight(decay=0.0001), 173 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/stage3_yolo11l_fold0_augmented/yolo11l_fold_0
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
optimizer: AdamW(lr=0.0001, momentum=0.937) with parameter groups 167 weight(decay=0.0), 174 weight(decay=0.0001), 173 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/stage3_yolo11l_fold0_augmented/yolo11l_fold_0
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      5.68G      1.379      3.528      1.858         32        640:   1%|          | 14/1608 [00:08<10:

In [ ]:
# Stage 4: Train RT-DETR-l and x on first fold of k_fold_cv_augmented

print("\n" + "="*100)
print("STAGE 4: Training RT-DETR-l and x on first fold of k_fold_cv_augmented")
print("="*100)

stage4_results = []
stage4_models = ['rtdetr-l', 'rtdetr-x']
fold_idx = 0  # First fold

# Check if dataset exists
if not os.path.exists(k_fold_cv_augmented_path):
    print(f"❌ Dataset path not found: {k_fold_cv_augmented_path}")
else:
    fold_dir = os.path.join(k_fold_cv_augmented_path, f"fold_{fold_idx}")
    data_yaml = os.path.join(fold_dir, "data.yaml")
    
    if os.path.exists(data_yaml):
        print(f"📁 Using fold {fold_idx}: {data_yaml}")
        
        # Train each RT-DETR model on fold 0
        for model_name in stage4_models:
            print(f"\n🚀 Starting {model_name} training on fold {fold_idx}...")
            
            result = train_model(
                model_type='rtdetr',
                model_name=model_name,
                data_yaml=data_yaml,
                fold_idx=fold_idx,
                experiment_name=f'stage4_{model_name}_fold0_augmented'
            )
            stage4_results.append(result)
    else:
        print(f"❌ Data YAML not found: {data_yaml}")
        for model_name in stage4_models:
            stage4_results.append({
                'model_name': model_name,
                'fold': fold_idx,
                'status': 'failed',
                'error': f'Data YAML not found: {data_yaml}'
            })

# Save Stage 4 results
stage4_file = save_results(stage4_results, "stage4")

# Summary
successful_stage4 = [r for r in stage4_results if r['status'] == 'completed']
print(f"\n📊 Stage 4 Summary: {len(successful_stage4)}/{len(stage4_models)} RT-DETR models completed successfully")

In [ ]:
# Final Summary and Analysis

print("\n" + "="*100)
print("COMPREHENSIVE TRAINING PIPELINE - FINAL SUMMARY")
print("="*100)

# Combine all results
all_results = stage1_results + stage2_results + stage3_results + stage4_results

# Overall statistics
total_trainings = len(all_results)
successful_trainings = len([r for r in all_results if r['status'] == 'completed'])
failed_trainings = total_trainings - successful_trainings

print(f"\n📊 OVERALL STATISTICS:")
print(f"Total trainings attempted: {total_trainings}")
print(f"Successful trainings: {successful_trainings}")
print(f"Failed trainings: {failed_trainings}")
print(f"Success rate: {(successful_trainings/total_trainings)*100:.1f}%")

# Stage-by-stage breakdown
print(f"\n📋 STAGE BREAKDOWN:")
stages = [
    ("Stage 1 (YOLO11n on k_fold_cv)", stage1_results, 5),
    ("Stage 2 (YOLO11n,s,m on k_fold_cv_augmented)", stage2_results, 15),
    ("Stage 3 (YOLO11l,x on fold 0)", stage3_results, 2),
    ("Stage 4 (RT-DETR-l,x on fold 0)", stage4_results, 2)
]

for stage_name, stage_results, expected_count in stages:
    successful = len([r for r in stage_results if r['status'] == 'completed'])
    print(f"  {stage_name}: {successful}/{expected_count} completed")

# Model performance summary (for successful trainings)
print(f"\n🏆 MODEL PERFORMANCE SUMMARY:")
successful_results = [r for r in all_results if r['status'] == 'completed' and r.get('best_fitness')]

if successful_results:
    # Group by model
    model_performance = {}
    for result in successful_results:
        model_name = result['model_name']
        if model_name not in model_performance:
            model_performance[model_name] = []
        model_performance[model_name].append(result['best_fitness'])
    
    # Calculate averages
    for model_name, fitness_values in model_performance.items():
        avg_fitness = sum(fitness_values) / len(fitness_values)
        max_fitness = max(fitness_values)
        min_fitness = min(fitness_values)
        count = len(fitness_values)
        
        print(f"  {model_name}: Avg={avg_fitness:.4f}, Max={max_fitness:.4f}, Min={min_fitness:.4f} ({count} runs)")

# Best performing models
if successful_results:
    best_model = max(successful_results, key=lambda x: x['best_fitness'])
    print(f"\n🥇 BEST PERFORMING MODEL:")
    print(f"  Model: {best_model['model_name']}")
    print(f"  Fold: {best_model['fold']}")
    print(f"  Fitness: {best_model['best_fitness']:.4f}")
    print(f"  Path: {best_model.get('model_path', 'N/A')}")

# Failed trainings summary
failed_results = [r for r in all_results if r['status'] == 'failed']
if failed_results:
    print(f"\n❌ FAILED TRAININGS:")
    for result in failed_results:
        print(f"  {result['model_name']} (Fold {result['fold']}): {result.get('error', 'Unknown error')}")

# Save comprehensive results
comprehensive_results = {
    'timestamp': datetime.datetime.now().isoformat(),
    'summary': {
        'total_trainings': total_trainings,
        'successful_trainings': successful_trainings,
        'failed_trainings': failed_trainings,
        'success_rate': (successful_trainings/total_trainings)*100
    },
    'stages': {
        'stage1': stage1_results,
        'stage2': stage2_results,
        'stage3': stage3_results,
        'stage4': stage4_results
    },
    'all_results': all_results
}

comprehensive_file = save_results(comprehensive_results, "comprehensive_pipeline")

print(f"\n✅ Comprehensive training pipeline completed!")
print(f"📁 All results saved to: {comprehensive_file}")
print(f"\n🔍 Check the 'runs/' directory for individual training outputs and models.")

## Training Pipeline Summary

This comprehensive training pipeline will train **24 models total**:

### Stage 1: YOLO11n Cross-Validation (5 models)
- YOLO11n on all 5 folds of `k_fold_cv`
- Provides baseline performance on original dataset

### Stage 2: Multi-Model Cross-Validation (15 models)
- YOLO11n, YOLO11s, YOLO11m on all 5 folds of `k_fold_cv_augmented`
- Tests effect of model size on augmented dataset

### Stage 3: Large YOLO Models (2 models)
- YOLO11l, YOLO11x on fold 0 of `k_fold_cv_augmented`
- Tests largest YOLO models for maximum performance

### Stage 4: RT-DETR Models (2 models)
- RT-DETR-l, RT-DETR-x on fold 0 of `k_fold_cv_augmented`
- Tests transformer-based detection architecture

### Training Configuration
- **Epochs**: 100 (with early stopping patience=20)
- **Image Size**: 640x640
- **Optimizer**: AdamW
- **Batch Sizes**: Automatically adjusted based on model size
  - Small models (n, s): batch=32
  - Medium models (m, rtdetr-l): batch=16  
  - Large models (l, x, rtdetr-x): batch=8

### Output Structure
All trained models will be saved in organized directories under `runs/`:
- `runs/stage1_yolo11n_k_fold_cv/`
- `runs/stage2_yolo11n_k_fold_cv_augmented/`
- `runs/stage2_yolo11s_k_fold_cv_augmented/`
- `runs/stage2_yolo11m_k_fold_cv_augmented/`
- `runs/stage3_yolo11l_fold0_augmented/`
- `runs/stage3_yolo11x_fold0_augmented/`
- `runs/stage4_rtdetr-l_fold0_augmented/`
- `runs/stage4_rtdetr-x_fold0_augmented/`

Results will be automatically saved to timestamped JSON files for analysis.